###### Capstone Project

###### Opening a New Shopping Mall in Mumbai, India
<ol>
    
   <li>Build a dataframe of neighborhoods in Mumbai, India by web scraping the data from Wikipedia page.</li>
   <li>Get the geographical coordinates of the neighborhoods.</li>
   <li>Obtain the venue data for the neighborhoods from Foursquare API.</li>
   <li>Explore and cluster the neighborhoods.</li>
   <li>Select the best cluster to open a new shopping mall.</li>
     </ol>

##### 1.Import libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import geocoder # to get coordinates

import requests # library to handle requests
from bs4 import BeautifulSoup # library to parse HTML and XML documents

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print("Libraries imported.")

Libraries imported.


##### 2. Scrap data from Wikipedia page into a DataFrame

In [2]:
# send the GET request
data = requests.get('https://en.wikipedia.org/wiki/Category:Suburbs_of_Mumbai').text

# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

neighborhoodList = []

# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)
    
mumbai_df = pd.DataFrame({"Neighborhood": neighborhoodList})

mumbai_df.head()

,Neighborhood
0,Andheri
1,Anushakti Nagar
2,Baiganwadi
3,Bandra
4,Bhandup


In [3]:
mumbai_df.shape

(42, 1)

##### 3. Get the geographical coordinates

In [4]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Mumbai, India'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

coords = [ get_latlng(neighborhood) for neighborhood in mumbai_df["Neighborhood"].tolist() ]

In [5]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [6]:
# merge the coordinates into the original dataframe
mumbai_df['Latitude'] = df_coords['Latitude']
mumbai_df['Longitude'] = df_coords['Longitude']

In [7]:
# check the neighborhoods and the coordinates
print(mumbai_df.shape)
mumbai_df.head()

(42, 3)


,Neighborhood,Latitude,Longitude
0,Andheri,19.118459,72.841763
1,Anushakti Nagar,19.042830,72.927340
2,Baiganwadi,19.062940,72.926630
3,Bandra,19.054370,72.840170
4,Bhandup,19.145560,72.948560


In [8]:
# save the DataFrame as CSV file
mumbai_df.to_csv("mumbai_df.csv", index=False)

##### 4. Create a map of Mumbai with neighborhoods superimposed on top

In [9]:
# get the coordinates of Mumbai
mumbai_address = 'Mumbai, India'

geolocator = Nominatim(user_agent="my-application")
location = geolocator.geocode(mumbai_address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Mumbai, India {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Mumbai, India 18.9387711, 72.8353355.


In [11]:
# create map of Mumbai using latitude and longitude values
map_mumbai = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(mumbai_df['Latitude'], mumbai_df['Longitude'], mumbai_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_mumbai)  
    
map_mumbai

In [12]:
# save the map as HTML file
map_mumbai.save('map_mumbai.html')

##### 5. Use the Foursquare API to explore the neighborhoods

In [35]:
# define Foursquare Credentials and Version
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 
CLIENT_SECRET:


In [14]:
radius = 2000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(mumbai_df['Latitude'], mumbai_df['Longitude'], mumbai_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(2717, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,Andheri,19.118459,72.841763,Merwans Cake shop,19.119300,72.845418,Bakery
1,Andheri,19.118459,72.841763,Naturals,19.111204,72.837255,Ice Cream Shop
2,Andheri,19.118459,72.841763,Shawarma Factory,19.124591,72.840398,Falafel Restaurant
3,Andheri,19.118459,72.841763,Radha Krishna Veg Restaurant,19.115130,72.843060,Indian Restaurant
4,Andheri,19.118459,72.841763,Starbucks Coffee : A Tata Alliance,19.114569,72.836205,Coffee Shop


###### Let's check how many venues were returned for each neighorhood

In [15]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
Andheri,100,100,100,100,100,100
Anushakti Nagar,17,17,17,17,17,17
Baiganwadi,8,8,8,8,8,8
Bandra,100,100,100,100,100,100
Bhandup,24,24,24,24,24,24
Borivali,100,100,100,100,100,100
Charkop,50,50,50,50,50,50
Chembur,100,100,100,100,100,100
Dahisar,67,67,67,67,67,67


######  Let's find out how many unique categories can be curated from all the returned venues

In [16]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 194 uniques categories.


In [17]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array([u'Bakery', u'Ice Cream Shop', u'Falafel Restaurant',
       u'Indian Restaurant', u'Coffee Shop', u'Pub', u'Pizza Place',
       u'Sandwich Place', u'Breakfast Spot', u'Chinese Restaurant',
       u'Multiplex', u'Juice Bar', u'Caf\xe9', u'American Restaurant',
       u'Diner', u'Snack Place', u'Seafood Restaurant',
       u'Maharashtrian Restaurant', u'Cocktail Bar',
       u'Gym / Fitness Center', u"Women's Store", u'BBQ Joint', u'Bar',
       u'Lounge', u'Fast Food Restaurant',
       u'Residential Building (Apartment / Condo)',
       u'Vegetarian / Vegan Restaurant', u'Spa', u'Electronics Store',
       u'Asian Restaurant', u'Smoke Shop', u'Food Truck', u'Liquor Store',
       u'Athletics & Sports', u'Fish Market', u'Tea Room', u'Park',
       u'Martial Arts Dojo', u'Hotel', u'Food', u'Plaza', u'Bus Station',
       u'Sports Bar', u'Food & Drink Shop', u'Music Venue',
       u'Hot Dog Joint', u'Tunnel', u'Fried Chicken Joint', u'Gym',
       u'Dessert Shop'], dtype=object)

In [18]:
# check if the results contain "Shopping Mall"
"Shopping Mall" in venues_df['VenueCategory'].unique()

True

##### 6. Analyze Each Neighborhood

In [19]:
# one hot encoding
mumbai_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
mumbai_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [mumbai_onehot.columns[-1]] + list(mumbai_onehot.columns[:-1])
mumbai_onehot = mumbai_onehot[fixed_columns]

print(mumbai_onehot.shape)
mumbai_onehot.head()

(2717, 195)


,Neighborhoods,Afghan Restaurant,American Restaurant,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Big Box Store,Bistro,Bookstore,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Building,Burger Joint,Burrito Place,Bus Station,Cafeteria,Café,Camera Store,Campground,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Auditorium,College Cafeteria,College Gym,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dhaba,Dim Sum Restaurant,Diner,Donut Shop,Electronics Store,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,General College & University,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Goan Restaurant,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Historic Site,History Museum,Hockey Arena,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Korean Restaurant,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Motorcycle Shop,Mountain,Movie Theater,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Outdoors & Recreation,Paper / Office Supplies Store,Park,Performing Arts Venue,Pharmacy,Pizza Place,Platform,Playground,Plaza,Pool,Pub,Punjabi Restaurant,Recreation Center,Residential Building (Apartment / Condo),Resort,Restaurant,Road,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Toy / Game Store,Track,Train Station,Travel & Transport,Tunnel,Vegetarian / Vegan Restaurant,Wine Bar,Wine Shop,Women's Store
0,Andheri,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Andheri,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Andheri,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Andheri,0

###### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [21]:
mumbai_grouped =mumbai_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(mumbai_grouped.shape)
mumbai_grouped

(41, 195)


,Neighborhoods,Afghan Restaurant,American Restaurant,Arcade,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Big Box Store,Bistro,Bookstore,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Building,Burger Joint,Burrito Place,Bus Station,Cafeteria,Café,Camera Store,Campground,Cheese Shop,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Auditorium,College Cafeteria,College Gym,Comedy Club,Comfort Food Restaurant,Concert Hall,Convenience Store,Convention Center,Cosmetics Shop,Coworking Space,Cricket Ground,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dhaba,Dim Sum Restaurant,Diner,Donut Shop,Electronics Store,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,General College & University,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Goan Restaurant,Golf Course,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Harbor / Marina,Hawaiian Restaurant,Historic Site,History Museum,Hockey Arena,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Irani Cafe,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Korean Restaurant,Liquor Store,Lounge,Maharashtrian Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Motorcycle Shop,Mountain,Movie Theater,Mughlai Restaurant,Multicuisine Indian Restaurant,Multiplex,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Outdoors & Recreation,Paper / Office Supplies Store,Park,Performing Arts Venue,Pharmacy,Pizza Place,Platform,Playground,Plaza,Pool,Pub,Punjabi Restaurant,Recreation Center,Residential Building (Apartment / Condo),Resort,Restaurant,Road,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,South Indian Restaurant,Spa,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park,Toy / Game Store,Track,Train Station,Travel & Transport,Tunnel,Vegetarian / Vegan Restaurant,Wine Bar,Wine Shop,Women's Store
0,Andheri,0.000000,0.010000,0.000000,0.000000,0.000000,0.020000,0.010000,0.010000,0.00,0.020000,0.0000,0.050000,0.00,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.010000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.0000,0.040000,0.000000,0.00,0.00,0.050000,0.000000,0.02,0.020000,0.00,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.020000,0.000000,0.020000,0.000000,0.0000,0.010000,0.00,0.000000,0.020000,0.00,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.040000,0.000000,0.00,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.010000,0.000000,0.040000,0.160000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.020000,0.00,0.010000,0.020000,0.010000,0.000000,0.01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.020000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.010000,0.000000,0.00000,0.050000,0.000000,0.000000,0.000000,0.000000,0.

In [22]:
len(mumbai_grouped[mumbai_grouped["Shopping Mall"] > 0])

18

######  Create a new DataFrame for Shopping Mall data only

In [24]:
malls = mumbai_grouped[["Neighborhoods","Shopping Mall"]]
malls.head()

,Neighborhoods,Shopping Mall
0,Andheri,0.000000
1,Anushakti Nagar,0.000000
2,Baiganwadi,0.000000
3,Bandra,0.000000
4,Bhandup,0.041667


###### 7. Cluster Neighborhoods
Run k-means to cluster the neighborhoods in Kuala Lumpur into 3 clusters.

In [25]:
# set number of clusters
kclusters = 3

mumbai_clustering = malls.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(mumbai_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 0, 0, 1, 2, 0, 2, 0, 2])

In [26]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
mumbai_merged = malls.copy()

# add clustering labels
mumbai_merged["Cluster Labels"] = kmeans.labels_

In [27]:
mumbai_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
mumbai_merged.head()

,Neighborhood,Shopping Mall,Cluster Labels
0,Andheri,0.000000,0
1,Anushakti Nagar,0.000000,0
2,Baiganwadi,0.000000,0
3,Bandra,0.000000,0
4,Bhandup,0.041667,1


In [28]:
# merge mumbai_grouped with mumbai_df to add latitude/longitude for each neighborhood
mumbai_merged = mumbai_merged.join(mumbai_df.set_index("Neighborhood"), on="Neighborhood")

print(mumbai_merged.shape)
mumbai_merged.head() # check the last columns!

(41, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Andheri,0.000000,0,19.118459,72.841763
1,Anushakti Nagar,0.000000,0,19.042830,72.927340
2,Baiganwadi,0.000000,0,19.062940,72.926630
3,Bandra,0.000000,0,19.054370,72.840170
4,Bhandup,0.041667,1,19.145560,72.948560


In [29]:
# sort the results by Cluster Labels
print(mumbai_merged.shape)
mumbai_merged.sort_values(["Cluster Labels"], inplace=True)
mumbai_merged

(41, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Andheri,0.000000,0,19.118459,72.841763
34,Tilak Nagar (Mumbai),0.000000,0,18.996160,72.852790
31,"Sion, Mumbai",0.000000,0,19.043410,72.863320
30,Shil Phata,0.000000,0,19.146580,73.040050
27,Mumbra,0.000000,0,19.167632,73.021408
25,Mira Road,0.000000,0,19.265674,72.870681
24,"Matharpacady, Mumbai",0.000000,0,18.950694,72.827268
23,Mankhurd,0.000000,0,19.048530,72.932220
22,Mahavir Nagar (Kandivali),0.000000,0,19.210940,72.841370
21,Kurla,0.000000,0,19.064980,72.880690


In [30]:
#create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(mumbai_merged['Latitude'], mumbai_merged['Longitude'], mumbai_merged['Neighborhood'], mumbai_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [31]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

##### 8. Examine Clusters

###### Cluster 0

In [32]:
mumbai_merged.loc[mumbai_merged['Cluster Labels'] == 0]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,Andheri,0.0,0,19.118459,72.841763
34,Tilak Nagar (Mumbai),0.0,0,18.996160,72.852790
31,"Sion, Mumbai",0.0,0,19.043410,72.863320
30,Shil Phata,0.0,0,19.146580,73.040050
27,Mumbra,0.0,0,19.167632,73.021408
25,Mira Road,0.0,0,19.265674,72.870681
24,"Matharpacady, Mumbai",0.0,0,18.950694,72.827268
23,Mankhurd,0.0,0,19.048530,72.932220
22,Mahavir Nagar (Kandivali),0.0,0,19.210940,72.841370
21,Kurla,0.0,0,19.064980,72.880690


###### Cluster 1

In [33]:
mumbai_merged.loc[mumbai_merged['Cluster Labels'] == 1]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
33,Thakur village,0.050847,1,19.21020,72.87541
26,Mulund,0.036585,1,19.17183,72.95565
4,Bhandup,0.041667,1,19.14556,72.94856
19,Kanjurmarg,0.071429,1,19.13138,72.93568
35,Uttan,0.045455,1,26.86634,80.93884


###### Cluster 2

In [34]:
mumbai_merged.loc[mumbai_merged['Cluster Labels'] == 2]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
37,Vikhroli,0.030303,2,19.111090,72.927810
38,Wadala,0.020408,2,19.017200,72.858170
32,"Sonapur, Bhandup",0.023810,2,19.163940,72.935440
36,Vashi,0.030000,2,19.084650,72.904810
13,Goregaon,0.020000,2,19.164550,72.849460
28,Pestom sagar,0.012500,2,19.070640,72.902170
5,Borivali,0.010000,2,19.229360,72.857510
7,Chembur,0.010000,2,19.062180,72.902410
9,Devipada,0.010870,2,19.224690,72.866050
39,Western Suburbs (Mumbai),0.012987,2,19.197010,72.827680


##### Observations:

Most of the shopping malls are concentrated in the  southern part of Mumbai city, with the highest number in cluster 1 and moderate number in cluster 2. On the other hand, cluster 0 has no shopping mall in the neighborhoods. This represents a great opportunity and high potential areas to open new shopping malls as there is no competition. Meanwhile, shopping malls in cluster 1 are likely suffering from intense competition due to oversupply and high concentration of shopping malls. From another perspective, this also shows that the oversupply of shopping malls mostly happened in the central area of the city, with the suburb area still have very few shopping malls. Therefore, this project recommends property developers to capitalize on these findings to open new shopping malls in neighborhoods in cluster 0 with no competition. Lastly, property developers are advised to avoid neighborhoods in cluster 2 which already have high concentration of shopping malls and suffering from intense competition.